In [1]:
import pandas as pd
import numpy as np
from datetime import date

In [2]:
if __name__ == "__main__":
    # set args
    api_fp = '../../admin/coingecko.txt'
    start_date = date(2015, 1, 1)
    end_date   = date(2023, 2, 1)
    base_url = "https://pro-api.coingecko.com/api/v3"
    asset_fp = "../data/raw/cmc_asset_universe.pkl"
    cw_fp    = "../data/raw/cmc_cw.pkl"
    panel_fp = "../data/raw/cmc_price_volume_mcap_panel.pkl"
    ex_fp = "../data/raw/cmc_exchange_panel.pkl"
    macro_fp = "../data/raw/cmc_macro.pkl"
    cw_new_old_col_mapping  = {'cmc_symbol': 'symbol_cmc',
                               'cmc_slug': 'slug_cmc',
                               'cmc_first_date': 'first_date_cmc',
                               'cmc_last_date': 'last_date_cmc'}

In [8]:

base_params = {'x_cg_pro_api_key': API_KEY}


In [11]:
# Set up API - FREE VERSION
base_url = "https://api.coingecko.com/api/v3"
base_params = {'x_cg_api_key': API_KEY}


In [13]:
# Test it is working
endpoint = '/ping'
final_url = base_url + endpoint
r = requests.get(final_url, params=base_params)
print(r.json()['gecko_says'])


(V3) To the Moon!


## (1) Determine universe of tokens

In [72]:
# Pull in the cmc data
cmc_df = pd.read_csv('../3-data/raw/cmc_token_universe.csv')


In [14]:
# Obtain list of all token identifiers
endpoint = '/coins/list'
final_url = base_url+endpoint
params = base_params.copy()
params['include_platform'] = 'false'
r = requests.get(final_url, params=params)
id_symbol_dict_list = r.json()
# id_symbol_dict_list is a list of dictionaries with coingecko ids, symbols, and names
len(id_symbol_dict_list)

12956

In [74]:
# Keep a copy in case I need it
master_id_symbol_dict_list = id_symbol_dict_list.copy()

In [75]:
# MATCH COINGECKO AND CMC TOKEN SYMBOLS

# Create new column to ensure CMC symbols are lower case
cmc_df['cmc_symbol_lower'] = cmc_df.cmc_symbol.str.lower()
cmc_symbols_lower = list(cmc_df.cmc_symbol_lower.values)

# Loop over all coingecko symbols to look for cmc symbol
for id_symbol_dict in id_symbol_dict_list:
    gecko_symbol = id_symbol_dict['symbol'].lower()
    if gecko_symbol in cmc_symbols_lower:
        print(gecko_symbol)
        gecko_id = id_symbol_dict['id']
        assert (cmc_df[cmc_df.cmc_symbol_lower == gecko_symbol].shape[0] >= 1)
        cmc_df.loc[cmc_df.cmc_symbol_lower == gecko_symbol, 'gecko_id'] = gecko_id

xcre
zoc
zch
zcn
zrx
zxc
zrx
man
100x
tshp
1337
1doge
1inch
1inch
1mil
1mt
safu
1-up
1wo
bvol
hedge
bnbhedge
ibvol
dogehedge
ethhedge
xrphedge
2chainlinks
2crz
2key
2lc
fit
algobull
altbull
bchbull
bsvbull
bull
bnbbull
adabull
linkbull
atombull
defibull
dogebull
drgnbull
eosbull
etcbull
ethbull
htbull
ltcbull
maticbull
midbull
okbbull
bullshit
sushibull
xautbull
xtzbull
tomobull
trxbull
xrpbull
bchbear
bsvbear
bear
bnbbear
adabear
linkbear
eosbear
etcbear
ethbear
ltcbear
sushibear
xautbear
xtzbear
trxbear
xrpbear
404
42
4art
kwatt
4stc
vgc
7e
7up
808ta
888
888
mph
8pay
exe
aave
abat
abat
abusd
abusd
adai
adai
aenj
aenj
aeth
ghst
aknc
aknc
alink
alink
amkr
amkr
asnx
asnx
atusd
atusd
aave
aab
abc
at
abc
abst
able
abs
abs
aaa
aca
aca
aca
acx
ace
aced
aced
ace
act
acxt
act
acm
ac
acoin
acs
acsi
acm
acu
acm
acu
aac
adaflect
admc
adm
adapad
adp
adax
adb
adx
add
adl
adx
adi
ara
ads
adt
aib
agld
ags
elf
ash
aens
aeon
aer
aergo
arnx
aet
atyne
aer
ae
aeth
ath
afen
fyn
afin
afr
att
afrox
edc
agar

dfa
fin
dfx
dzi
defx
finix
pie
dpi
cwap
xgm
dfc
dss
dfs
dsc
dfi
sta
defit
defi5
isla
fiwa
dyp
deflct
dfy
dg
dac
dac
meme
degen
dgvc
dego
dct
heroes
dhv
dehub
dio
dip
djv
dek
deku
ira
dotx
dpy
delta
delta
dhc
dlta
dusd
dmod
dmd
d
dnz
dent
dcn
don
depay
dep
deq
derc
drc
deri
ddx
dvx
dero
des
ds
dth
deus
dem
deva
dvt
eve
dev
life
dev
dexa
bro
dxc
dexe
dxf
dxg
dexi
dex
dxt
kit
dxl
dexm
dextf
dexg
dext
dxc
dxo
xfi
buidl
xbtc
xeth
df
dfsocial
dfnd
dft
dfx
dfyn
dgp
dbc
dht
dusd
dtop
dia
dmd
dbz
dnd
dhc
dhold
love
dpt
diamond
diah
dibs
dice
did
yfiii
dig
digg
digi
dgb
dgcl
ddr
ddrst
ddrt
digs
dft
xdb
dgc
dusd
dfs
dmb
xdn
dp
dzar
drc
dtx
dgtx
dgd
dgx
dime
dmtr
dms
dina
dcy
din
dingo
dingo
dink
dino
dino
dino
dnxc
dion
dip
dip
dirty
ddos
dnt
distx
ditto
diver
dfi
divi
dpt
dka
duck
dmt
dmd
dmme
dmg
dmt
dna
dna
dxct
dnf
dobe
dock
dms
dodo
drm
dog
dogdefi
dogec
doge
doge2
dogecola
dogefather
dogefi
dogegf
dogex
elon
elon
dogo
dogo
don
dogepepsi
doges
dtf
doget
spacexdoge
dwr
dogy
dinu
dzoo
doggy
ds

mpay
mpl
mrs
meta
power
shell
shield
msd
mts
msc
met
metauniverse
mvc
mdao
dna
etp
mvi
meta
play
meta
mwbtc
mwg
meta
mtrg
emtrg
mtr
mthd
mts
metis
metric
met
met
mtlx
mex
mexc
mgc
mia
mib
mbc
mbtc
mcc
micro
mgp
micro
amm
microshib
mct
midas
mdo
mas
miks
mvc
milf
mil
milk
milk
mlk
mb
milk
mm
mm
mimo
mfc
milo
swarm
mimatic
mwc
mimo
mimo
mina
mnc
mdc
mnd
mnd
min
minds
mai
mb
mcr
mnr
mne
mg
gold
mnm
mini
mbtc
mada
minidoge
minifloki
minifootball
minisoccer
miniusdc
mnx
max
mint
mint
mint
mint
hub
mint
mintys
mql
mbaba
mamzn
maapl
mbtc
meth
mfb
mgoogl
mqqq
miau
mslv
mmsft
mnflx
mtsla
mtwtr
muso
mir
msb
msk
me
helios
mist
mist
mith
mis
xin
mix
some
mxt
mxf
mch
mlm
mmaon
mmo
mcash
mm
mbx
mofi
mob
mcpc
mgo
mbn
molk
mob
mitx
mobi
mot
mobi
mbox
mochi
moma
mcm
mochi
mok
mod
modx
modex
aim
mobi
mda
mogx
stars
mojo
mof
momat
xmm
mtm
mmon
key
momo
momo
mona
mona
mona
xmr
xmc
xmv
moneta
mue
mcn
mth
msd
mnc
mswap
imt
money
monk
mns
mcash
moni
mat
mss
mons
moochii
moon
moonarch
mbf
mbet
moon
moon
moone

shake
spacetoast
space
space
safe
spank
spark
spark
sprkl
srk
sfuel
spk
spa
300
spn
sparta
spa
spc
alias
sxdt
xspc
sxut
spt
xsm
spec
scs
sms
spell
spell
spnd
spa
usds
sphr
sat
sphri
sph
spx
sph
spice
spice
spice
spdr
spike
spin
spin
spd
spin
spr
ghost
care
spirit
sps
shopx
spkl
boo
spore
spo
snl
spt
spt
sprink
sprts
spex
nuts
sact
ssv
stbz
set
susd
sta
sdt
usds
stack
stack
dsla
stacy
fis
scc
sdt
steth
stzen
stfiro
steth
smoon
xsn
str
stkr
sh
stt
stnd
tst
sac
atlas
polis
atlas
star
suni
bot
stc
stc
foxx
stars
stark
stars
starl
ore
smon
iov
sss
starship
stars
stars
stt
stfi
vest
str
eurs
str
sta
snt
stax
safu
sbf
steak
steak
xst
steel
steem
sbd
xlm
xld
tryon
xlpg
xla
step
step
hero
step
stn
sdfi
spd
stbu
ges
stn
stonk
stonk
sbf
a
stopelon
sana
sov
stq
storj
stmx
storm
srx
story
stx
stpt
strax
stos
stpl
data
strk
strx
stri
strong
bull
shnd
shmn
shx
stf
sgb
sub
sub
sfx
success
xsr
sugar
sugar
sugar
skt
suku
sum
sum
sumo
sum
sun
snc
sunder
suni
sunny
sun
superbid
super
super
scv
super
sloki

In [76]:
# MATCH THE UNMATCHED TOKENS ON GECKO NAME AND CMC SLUG

# Create new column to ensure CMC slugs are lower case
cmc_df['cmc_slug_lower'] = cmc_df.cmc_slug.str.lower()
cmc_slugs_lower = list(cmc_df[cmc_df.gecko_id.isnull()].cmc_slug_lower.values)

# Loop over all cmc slugs that didn't match to look for cmc symbol
for cmc_slug in cmc_slugs_lower:
    for id_symbol_dict in id_symbol_dict_list:
        gecko_name = id_symbol_dict['name'].lower()
        if cmc_slug == gecko_name:
            gecko_id = id_symbol_dict['id'].lower()
            print(gecko_name)
            assert (cmc_df[cmc_df.cmc_slug_lower == gecko_name].shape[0] >= 1)
            cmc_df.loc[cmc_df.cmc_slug_lower == gecko_name, 'gecko_id'] = gecko_id


alphacoin
corgicoin
coin
shield
cryptonite
moneta
euphoria
paycoin
gems
omega
empyrean
greed
vibranium
freedomcoin
upcoin
doubloon
vsync
eaglecoin
traxia
firmachain
unicrypt
sun
moon
azzure
nhbtc
impulseven
moondao
wenmoon
dogemoon
weentar
aurum
mybricks
defpace
gameology
honeypad


In [77]:
# MATCH THE UNMATCHED TOKENS ON GECKO NAME AND CMC NAME

# Create new column to ensure CMC slugs are lower case
cmc_df['cmc_name_lower'] = cmc_df.name.str.lower()
cmc_names_lower = list(cmc_df[cmc_df.gecko_id.isnull()].cmc_slug_lower.values)

# Loop over all cmc slugs that didn't match to look for cmc symbol
for cmc_name in cmc_names_lower:
    for id_symbol_dict in id_symbol_dict_list:
        gecko_name = id_symbol_dict['name'].lower()
        if cmc_name == gecko_name:
            gecko_id = id_symbol_dict['id'].lower()
            print(gecko_name)
            assert (cmc_df[cmc_df.cmc_name_lower == gecko_name].shape[0] >= 1)
            cmc_df.loc[cmc_df.cmc_name_lower == gecko_name, 'gecko_id'] = gecko_id


In [78]:
# LIST OUT THE UNMATCHED TOKENS OF HIGH MCAP, WHICH LOOK TO BE NOT NECESSARY
cmc_unmatched_df = cmc_df[cmc_df.gecko_id.isnull()]
cmc_mcap_df = pd.read_csv('../3-data/raw/cmc_price_vol_mcap_panel.csv')
cmc_mcap_df = cmc_mcap_df[cmc_mcap_df.cmc_id.isin(cmc_unmatched_df.cmc_id.values)]
missing_sig_tokens = np.unique(cmc_mcap_df[(cmc_mcap_df.usd_mcap > 1000000) & (cmc_mcap_df.usd_volume_24h > 7000)].cmc_id.values)
cmc_df[cmc_df.cmc_id.isin(list(missing_sig_tokens))].name.values

array(['BitShares PTS', 'LottoCoin', 'YbCoin', 'TeslaCoin', 'UltraCoin',
       'Blakecoin', 'QubitCoin', 'HunterCoin', 'CacheCoin', 'DopeCoin',
       'MAZA', 'UnionCoin', 'Rubycoin', 'Bela', 'FlutterCoin',
       'StabilityShares', 'Canada eCoin', 'Energycoin', 'Qora',
       'Scotcoin', 'BoostCoin', 'BitQuark', 'Coin2.1', 'FuelCoin', 'Kore',
       'GoldReserve', 'BritCoin', 'BitcoinDark', 'Titcoin', 'VirtualCoin',
       'Virtacoin', 'Joincoin', 'Storjcoin X', 'Donu', 'BilShares',
       'Opal', 'FoldingCoin', 'bitUSD', 'bitBTC', 'Sterlingcoin', 'Magi',
       'Banx', 'Jinn', 'Dashcoin', 'MMNXT', 'GanjaCoin', 'TileCoin',
       'Blitzcash', 'Bytecent', 'Pangea Poker', 'PayCoin', 'vTorrent',
       'Dotcoin', 'X-Coin', 'Colossuscoin V2', 'Pura', '8Bit', 'Neutron',
       'SounDAC', 'ParkByte', 'TRMB', 'Synergy', 'FujiCoin',
       'The Cypherfunks', 'GravityCoin', 'AnarchistsPrime', 'Capricoin+',
       'TransferCoin', 'AmsterdamCoin', 'Eurocoin', 'Shift',
       'BitCrystals', 'Pro

In [79]:
# CUT DOWN TOKEN UNIVERSE TO TOKENS ABOVE MCAP AND VOLUME THRESHOLDS

# Obtain CMC IDs that correspond to non-missing gecko id
cmc_ids_mapped_to_gecko = cmc_df[~cmc_df.gecko_id.isnull()].cmc_id.values

# Cut down the mcap data to just these CMC IDs
cmc_mcap_df = pd.read_csv('../3-data/raw/cmc_price_vol_mcap_panel.csv')
cmc_mcap_df = cmc_mcap_df[cmc_mcap_df.cmc_id.isin(cmc_ids_mapped_to_gecko)]

# Merge on the total mcap data
macro_df = pd.read_csv('../3-data/raw/cmc_macro_timeseries.csv')
macro_df = macro_df[['date', 'total_market_cap']]
cmc_mcap_df = cmc_mcap_df.merge(macro_df,
                                on='date',
                                how='inner',
                                validate='many_to_one')

# Create the mcap threshold and drop tokens below mcap and and volume thresholds
cmc_mcap_df['mcap_threshold'] = cmc_mcap_df.total_market_cap * 0.0001
cmc_mcap_df = cmc_mcap_df[cmc_mcap_df.usd_mcap > cmc_mcap_df.mcap_threshold]
cmc_mcap_df = cmc_mcap_df[cmc_mcap_df.usd_volume_24h >= 100000]

# Obtain the cmc ids of the tokens above the threshold
cmc_ids_above_mcap_volume_threshold = np.unique(cmc_mcap_df.cmc_id.values)

# Obtain the corresponding gecko ids above threshold
gecko_id_above_mcap_volume_threshold = np.unique(cmc_df[cmc_df.cmc_id.isin(cmc_ids_above_mcap_volume_threshold)].gecko_id.values)


In [80]:
# Create object for the unique gecko id's to go gather data for
gecko_id_universe = gecko_id_above_mcap_volume_threshold.copy()

In [91]:
# Save the crosswalk for use
cmc_df = cmc_df[['cmc_id', 'cmc_symbol', 'name', 'cmc_slug', 'gecko_id']]
cmc_df = cmc_df[cmc_df.gecko_id.isin(gecko_id_above_mcap_volume_threshold)]
cmc_df.to_pickle('../3-data/raw/coingecko_cw_to_cmc.pkl')

## (2) Pull price, market cap, volume data

In [17]:
# Initialize dictionary to hold results
price_mcap_vol_dict = {'gecko_id': [],
                       'date': [],
                       'stat': [],
                       'value': []}

# Initialize counter
i = 0

# Loop over all the gecko ids
for token_id in gecko_id_universe:
    print(token_id)
    print('Completed ' + str(np.round(i/len(gecko_id_universe) * 100, 2)) + '% of the tokens.')

    # Set up the call
    endpoint = '/coins/' + token_id + '/market_chart/range'
    final_url = base_url+endpoint
    params = base_params.copy()
    params['id'] = token_id
    params['vs_currency'] = 'usd'
    params['from'] = str(int(dt.datetime(year=2015, month=1, day=1).timestamp()))
    params['to'] = str(int(dt.datetime(year=2022, month=1, day=7).timestamp()))

    # Make the call while checking it is successful
    nb_tries = 3
    while True:
        nb_tries -= 1
        try:
            r = requests.get(final_url, params=params, timeout=2)
            if r.status_code == 200:
                r_json = r.json()
                break
            else:
                if nb_tries <= 0:
                    assert(1==0),('some other error')
                pass
        except (ConnectionError, requests.exceptions.Timeout) as err:
            if nb_tries <= 0:
                raise err
            else:
                print('connection or timeout error')
                time.sleep(2)

    # Extract the information
    for price in r_json['prices']:
        price_mcap_vol_dict['gecko_id'].append(token_id)
        price_mcap_vol_dict['date'].append(dt.datetime.fromtimestamp(int(price[0] / 1000)).strftime("%Y-%m-%d"))
        price_mcap_vol_dict['stat'].append('usd_per_token')
        price_mcap_vol_dict['value'].append(price[1])

    for mcap in r_json['market_caps']:
        price_mcap_vol_dict['gecko_id'].append(token_id)
        price_mcap_vol_dict['date'].append(dt.datetime.fromtimestamp(int(mcap[0] / 1000)).strftime("%Y-%m-%d"))
        price_mcap_vol_dict['stat'].append('usd_mcap')
        price_mcap_vol_dict['value'].append(mcap[1])

    for volume in r_json['total_volumes']:
        price_mcap_vol_dict['gecko_id'].append(token_id)
        price_mcap_vol_dict['date'].append(dt.datetime.fromtimestamp(int(volume[0] / 1000)).strftime("%Y-%m-%d"))
        price_mcap_vol_dict['stat'].append('usd_volume_24h')
        price_mcap_vol_dict['value'].append(volume[1])

    # Space out the calls and increment counter
    time.sleep(0.12)
    i += 1
    print('\n')

0x-wormhole
Completed 0.0% of the tokens.


12ships
Completed 0.1% of the tokens.


1inch-wormhole
Completed 0.19% of the tokens.


8x8-protocol
Completed 0.29% of the tokens.


aave-wormhole
Completed 0.38% of the tokens.


aca-token
Completed 0.48% of the tokens.


achain
Completed 0.58% of the tokens.


acute-angle-cloud
Completed 0.67% of the tokens.


advanced-internet-block
Completed 0.77% of the tokens.


adventure-gold
Completed 0.87% of the tokens.


aeon
Completed 0.96% of the tokens.
connection or timeout error


aergo
Completed 1.06% of the tokens.


aeron
Completed 1.15% of the tokens.


aeternity
Completed 1.25% of the tokens.


agoras-currency-of-tau
Completed 1.35% of the tokens.
connection or timeout error


agrello
Completed 1.44% of the tokens.


agro-global
Completed 1.54% of the tokens.


aichain
Completed 1.64% of the tokens.


aidos-kuneen
Completed 1.73% of the tokens.


aion
Completed 1.83% of the tokens.
connection or timeout error


akash-network
Completed 1.



circuits-of-value
Completed 16.94% of the tokens.


cloakcoin
Completed 17.04% of the tokens.


clover-finance
Completed 17.13% of the tokens.


cnn
Completed 17.23% of the tokens.
connection or timeout error


coin98
Completed 17.32% of the tokens.


coinmetro
Completed 17.42% of the tokens.


compound-wormhole
Completed 17.52% of the tokens.


concentrated-voting-power
Completed 17.61% of the tokens.


concierge-io
Completed 17.71% of the tokens.


conflux-token
Completed 17.81% of the tokens.


constellation-labs
Completed 17.9% of the tokens.


contentos
Completed 18.0% of the tokens.


convex-finance-wormhole
Completed 18.09% of the tokens.


cortex
Completed 18.19% of the tokens.


cosmo-coin
Completed 18.29% of the tokens.


cosmos
Completed 18.38% of the tokens.


coti
Completed 18.48% of the tokens.


counos-coin
Completed 18.58% of the tokens.


counosx
Completed 18.67% of the tokens.


counterparty
Completed 18.77% of the tokens.
connection or timeout error


covicoin
Comp

connection or timeout error


fedoracoin
Completed 33.69% of the tokens.


fei-usd
Completed 33.78% of the tokens.


fetch-ai
Completed 33.88% of the tokens.


filecoin
Completed 33.97% of the tokens.


fitmin
Completed 34.07% of the tokens.


flash-token-2
Completed 34.17% of the tokens.


flower-solana
Completed 34.26% of the tokens.


fnb-protocol
Completed 34.36% of the tokens.


fortuna-sittard-fan-token
Completed 34.46% of the tokens.


frax-share-wormhole
Completed 34.55% of the tokens.


frax-wormhole
Completed 34.65% of the tokens.


freedomcoin
Completed 34.74% of the tokens.


freerossdao
Completed 34.84% of the tokens.


fsn
Completed 34.94% of the tokens.


ftx-wormhole
Completed 35.03% of the tokens.


fuel-token
Completed 35.13% of the tokens.


funfair
Completed 35.23% of the tokens.


futurescash
Completed 35.32% of the tokens.


futureswap
Completed 35.42% of the tokens.


fx-coin
Completed 35.51% of the tokens.


fxt-token
Completed 35.61% of the tokens.


gala-wormh



mdex
Completed 50.72% of the tokens.


medibloc
Completed 50.82% of the tokens.


medicalchain
Completed 50.91% of the tokens.


megacoin
Completed 51.01% of the tokens.


melon
Completed 51.11% of the tokens.


memetic
Completed 51.2% of the tokens.


mercury
Completed 51.3% of the tokens.


meta-capital
Completed 51.4% of the tokens.


metacoin
Completed 51.49% of the tokens.


metadogeswap
Completed 51.59% of the tokens.


metaface
Completed 51.68% of the tokens.


metahash
Completed 51.78% of the tokens.


metal
Completed 51.88% of the tokens.


metaverse-dualchain-network-architecture
Completed 51.97% of the tokens.


metaverse-etp
Completed 52.07% of the tokens.


metazuckzilla
Completed 52.17% of the tokens.


metis-token
Completed 52.26% of the tokens.


mexc-token
Completed 52.36% of the tokens.


milk-alliance
Completed 52.45% of the tokens.


mim
Completed 52.55% of the tokens.


mimblewimblecoin
Completed 52.65% of the tokens.


mina-protocol
Completed 52.74% of the token



qtum
Completed 68.24% of the tokens.


quant-network
Completed 68.33% of the tokens.


quantstamp
Completed 68.43% of the tokens.


quantum-resistant-ledger
Completed 68.53% of the tokens.


quark
Completed 68.62% of the tokens.
connection or timeout error


quark-chain
Completed 68.72% of the tokens.


qubit
Completed 68.82% of the tokens.


quickswap-wormhole
Completed 68.91% of the tokens.


radicle
Completed 69.01% of the tokens.


radio-caca
Completed 69.1% of the tokens.


raiden-network
Completed 69.2% of the tokens.


rally-2
Completed 69.3% of the tokens.


ramp
Completed 69.39% of the tokens.


rari-governance-token-wormhole
Completed 69.49% of the tokens.


rarible
Completed 69.59% of the tokens.


raven-dark
Completed 69.68% of the tokens.


ravencoin
Completed 69.78% of the tokens.


raydium-wormhole
Completed 69.87% of the tokens.


realio-network
Completed 69.97% of the tokens.


realtract
Completed 70.07% of the tokens.


red-pulse
Completed 70.16% of the tokens.


re



token-pocket
Completed 85.37% of the tokens.


tokenomy
Completed 85.47% of the tokens.


tokenpay
Completed 85.56% of the tokens.


tokocrypto
Completed 85.66% of the tokens.


tokonft
Completed 85.76% of the tokens.


tomochain
Completed 85.85% of the tokens.


top-network
Completed 85.95% of the tokens.


transhuman-coin
Completed 86.04% of the tokens.


treasure-sl
Completed 86.14% of the tokens.


tribe-wormhole
Completed 86.24% of the tokens.


trillium
Completed 86.33% of the tokens.
connection or timeout error


trinity-network-credit
Completed 86.43% of the tokens.


triumphx
Completed 86.53% of the tokens.


tron-bsc
Completed 86.62% of the tokens.


tronbetdice
Completed 86.72% of the tokens.


troy
Completed 86.81% of the tokens.


true-chain
Completed 86.91% of the tokens.


true-usd
Completed 87.01% of the tokens.


truefi
Completed 87.1% of the tokens.


trust-wallet-token
Completed 87.2% of the tokens.


trustswap
Completed 87.3% of the tokens.


tt-token
Completed 87

In [18]:
# Convert dictionary to cleaned up dataframe
gecko_price_mcap_vol_df = pd.DataFrame(price_mcap_vol_dict)
gecko_price_df = gecko_price_mcap_vol_df[gecko_price_mcap_vol_df.stat == 'usd_per_token']
gecko_price_df = gecko_price_df[~gecko_price_df.duplicated(subset=['gecko_id', 'date'])]
gecko_mcap_df  = gecko_price_mcap_vol_df[gecko_price_mcap_vol_df.stat == 'usd_mcap']
gecko_mcap_df  = gecko_mcap_df[~gecko_mcap_df.duplicated(subset=['gecko_id', 'date'])]
gecko_vol_df   = gecko_price_mcap_vol_df[gecko_price_mcap_vol_df.stat == 'usd_volume_24h']
gecko_vol_df   = gecko_vol_df[~gecko_vol_df.duplicated(subset=['gecko_id', 'date'])]
gecko_price_df = pd.pivot(gecko_price_df, index=['date', 'gecko_id'], columns='stat', values='value').reset_index()
gecko_mcap_df  = pd.pivot(gecko_mcap_df, index=['date', 'gecko_id'], columns='stat', values='value').reset_index()
gecko_vol_df   = pd.pivot(gecko_vol_df, index=['date', 'gecko_id'], columns='stat', values='value').reset_index()
gecko_df = gecko_price_df.merge(gecko_mcap_df,
                                on=['date', 'gecko_id'],
                                how='inner',
                                validate='one_to_one')
gecko_df = gecko_df.merge(gecko_vol_df,
                          on=['date', 'gecko_id'],
                          how='inner',
                          validate='one_to_one')

## (3) Pull coingecko unique covariate data for each coin's history

In [19]:
# Update the gecko universe for coins that had price data
gecko_id_universe_valid = np.unique(gecko_df.gecko_id.values)

print(len(gecko_id_universe_valid)/len(gecko_id_universe))

0.9027911453320501


In [28]:
# Initialize a dictionary for the results
gecko_covars_dict = {'date':[],
                     'gecko_id': [],
                     'twitter_followers': [],
                     'reddit_average_posts_48h': [],
                     'reddit_average_comments_48h': [],
                     'reddit_subscribers': [],
                     'reddit_accounts_active_48h': [],
                     'forks': [],
                     'stars': [],
                     'subscribers': [],
                     'total_issues': [],
                     'closed_issues': [],
                     'pull_requests_merged': [],
                     'pull_request_contributors': [],
                     'code_additions_4_weeks': [],
                     'code_deletions_4_weeks': [],
                     'commit_count_4_weeks': [],
                     'alexa_rank': []}

In [32]:
# Loop over all the tokens
i=0
for gecko_id in gecko_id_universe_valid[i:]:
    # Update where we at
    print(gecko_id)
    print('Completed ' + str(np.round(i/len(gecko_id_universe_valid) * 100, 2)) + '% of the tokens.')
    
    # Extract the earliest date that we have price data for that token
    token_dates = gecko_df[gecko_df.gecko_id == gecko_id].date.values
    token_dates = [dt.datetime.strptime(date, "%Y-%m-%d").date() for date in token_dates]
    first_date  = np.min(token_dates)
    first_date  = first_date.strftime(format="%d-%m-%Y")

    # Build list of dates to pull
    dates = list(pd.date_range(first_date, '07-01-2022', freq='D').strftime('%d-%m-%Y'))

    # Loop over all the dates
    for date_to_call in dates:
        # Set up the call
        endpoint       = '/coins/' + gecko_id + '/history'
        final_url      = base_url+endpoint
        params         = base_params.copy()
        params['id']   = gecko_id 
        params['date'] = date_to_call

        # Make the call while checking it is successful
        nb_tries = 5
        while True:
            nb_tries -= 1
            try:
                r = requests.get(final_url, params=params, timeout=2)
                if r.status_code == 200:
                    r_json = r.json()
                    break
                else:
                    if nb_tries <= 0:
                        assert(1==0),('some other error')
                    time.sleep(2)
                    pass
            except (ConnectionError, requests.exceptions.Timeout) as err:
                if nb_tries <= 0:
                    raise err
                else:
                    print('connection or timeout error')
                    time.sleep(5)

        # Add data to dictionary
        date_of_call = date_to_call[-4:] + '-' + date_to_call[3:5] + '-' + date_to_call[:2]
        gecko_covars_dict['date'].append(date_of_call)
        gecko_covars_dict['gecko_id'].append(gecko_id)

        if 'community_data' in r_json.keys():
            gecko_covars_dict['twitter_followers'].append(r_json['community_data']['twitter_followers'])
            gecko_covars_dict['reddit_average_posts_48h'].append(r_json['community_data']['reddit_average_posts_48h'])
            gecko_covars_dict['reddit_average_comments_48h'].append(r_json['community_data']['reddit_average_comments_48h'])
            gecko_covars_dict['reddit_subscribers'].append(r_json['community_data']['reddit_subscribers'])
            gecko_covars_dict['reddit_accounts_active_48h'].append(r_json['community_data']['reddit_accounts_active_48h'])
        else:
            gecko_covars_dict['twitter_followers'].append(None)
            gecko_covars_dict['reddit_average_posts_48h'].append(None)
            gecko_covars_dict['reddit_average_comments_48h'].append(None)
            gecko_covars_dict['reddit_subscribers'].append(None)
            gecko_covars_dict['reddit_accounts_active_48h'].append(None)

        if 'developer_data' in r_json.keys():
            gecko_covars_dict['forks'].append(r_json['developer_data']['forks'])
            gecko_covars_dict['stars'].append(r_json['developer_data']['stars'])
            gecko_covars_dict['subscribers'].append(r_json['developer_data']['subscribers'])
            gecko_covars_dict['total_issues'].append(r_json['developer_data']['total_issues'])
            gecko_covars_dict['closed_issues'].append(r_json['developer_data']['closed_issues'])
            gecko_covars_dict['pull_requests_merged'].append(r_json['developer_data']['pull_requests_merged'])
            gecko_covars_dict['pull_request_contributors'].append(r_json['developer_data']['pull_request_contributors'])
            gecko_covars_dict['code_additions_4_weeks'].append(r_json['developer_data']['code_additions_deletions_4_weeks']['additions'])
            gecko_covars_dict['code_deletions_4_weeks'].append(r_json['developer_data']['code_additions_deletions_4_weeks']['deletions'])
            gecko_covars_dict['commit_count_4_weeks'].append(r_json['developer_data']['commit_count_4_weeks'])
        else:
            gecko_covars_dict['forks'].append(None)
            gecko_covars_dict['stars'].append(None)
            gecko_covars_dict['subscribers'].append(None)
            gecko_covars_dict['total_issues'].append(None)
            gecko_covars_dict['closed_issues'].append(None)
            gecko_covars_dict['pull_requests_merged'].append(None)
            gecko_covars_dict['pull_request_contributors'].append(None)
            gecko_covars_dict['code_additions_4_weeks'].append(None)
            gecko_covars_dict['code_deletions_4_weeks'].append(None)
            gecko_covars_dict['commit_count_4_weeks'].append(None)

        if 'public_interest_stats' in r_json.keys():
            gecko_covars_dict['alexa_rank'].append(r_json['public_interest_stats']['alexa_rank'])
        else:
            gecko_covars_dict['alexa_rank'].append(None)
        
        # Space out the calls and increment counter
        time.sleep(0.1)
    
    # Update the counter for the tokens
    i += 1
    print('\n')


selfkey
Completed 74.73% of the tokens.
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error


sentinel-group
Completed 74.84% of the tokens.
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or t

connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error


somesing
Completed 77.08% of the tokens.
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error


sonm
Completed 77.19% of the tokens.
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout 

connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error


step-hero
Completed 79.32% of the tokens.
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error


st

connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error


swftcoin
Completed 81.66% of the tokens.
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error


swissborg
Completed 81.77% of the tokens.
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or tim

connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error


the-parallel
Completed 83.8% of the tokens.
connection or timeout error


thekey
Completed 83.9% of the tokens.
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error


theta-token
Comp

...

connection or timeout error


utrust
Completed 89.23% of the tokens.
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error


v-id-blockchain
Completed 89.34% of the tokens.
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or

connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error


vibe-token
Completed 91.15% of the tokens.
connection or timeout error
connection or timeout error
connection or timeout error


viberate
Completed 91.26% of the tokens.
connection or ti

connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error


white-whale
Completed 93.39% of the tokens.
connection or timeout error
connection or timeout error


whitecoin
Completed 93.5% of the tokens.
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or t

connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error


xtrabytes
Completed 96.59% of the tokens.
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error


xverse
Completed 96.7% of the tokens.
connection or timeout error
connection or timeout error
connection or timeout error


xyo-network
Completed 96.8% of the tokens.
connection or timeout error
connection or timeout error
con

connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error


zeepin
Completed 98.72% of the tokens.
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error


zelcash
Completed 98.83% of the tokens.
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout error
connection or timeout

In [60]:
# Convert to a dataframe
gecko_covars_df = pd.DataFrame(gecko_covars_dict)
gecko_covars_df = gecko_covars_df[~gecko_covars_df.duplicated(subset=['date', 'gecko_id'])]

In [63]:
# Merge the two dataframes together
df = gecko_df.merge(gecko_covars_df,
                    on=['date', 'gecko_id'],
                    how='outer',
                    validate='one_to_one')

In [67]:
# Save the data
df.to_csv('../3-data/raw/coingecko_panel.csv', index=False)

## (3) Pull BTC and ETH data

In [18]:
gecko_ids = ['bitcoin', 'ethereum']

In [19]:
# Initialize dictionary to hold results
price_mcap_vol_dict = {'gecko_id': [],
                       'date': [],
                       'stat': [],
                       'value': []}

# Initialize counter
i = 0

# Loop over all the gecko ids
for token_id in gecko_ids:
    print(token_id)
    
    # Set up the call
    endpoint = '/coins/' + token_id + '/market_chart/range'
    final_url = base_url+endpoint
    params = base_params.copy()
    params['id'] = token_id
    params['vs_currency'] = 'usd'
    params['from'] = str(int(dt.datetime(year=2015, month=12, day=1).timestamp()))
    params['to'] = str(int(dt.datetime(year=2022, month=8, day=13).timestamp()))

    # Make the call while checking it is successful
    nb_tries = 3
    while True:
        nb_tries -= 1
        try:
            r = requests.get(final_url, params=params, timeout=2)
            if r.status_code == 200:
                r_json = r.json()
                break
            else:
                if nb_tries <= 0:
                    assert(1==0),('some other error')
                pass
        except (ConnectionError, requests.exceptions.Timeout) as err:
            if nb_tries <= 0:
                raise err
            else:
                print('connection or timeout error')
                time.sleep(2)

    # Extract the information
    for price in r_json['prices']:
        price_mcap_vol_dict['gecko_id'].append(token_id)
        price_mcap_vol_dict['date'].append(dt.datetime.fromtimestamp(int(price[0] / 1000)).strftime("%Y-%m-%d"))
        price_mcap_vol_dict['stat'].append('usd_per_token')
        price_mcap_vol_dict['value'].append(price[1])

    for mcap in r_json['market_caps']:
        price_mcap_vol_dict['gecko_id'].append(token_id)
        price_mcap_vol_dict['date'].append(dt.datetime.fromtimestamp(int(mcap[0] / 1000)).strftime("%Y-%m-%d"))
        price_mcap_vol_dict['stat'].append('usd_mcap')
        price_mcap_vol_dict['value'].append(mcap[1])

    for volume in r_json['total_volumes']:
        price_mcap_vol_dict['gecko_id'].append(token_id)
        price_mcap_vol_dict['date'].append(dt.datetime.fromtimestamp(int(volume[0] / 1000)).strftime("%Y-%m-%d"))
        price_mcap_vol_dict['stat'].append('usd_volume_24h')
        price_mcap_vol_dict['value'].append(volume[1])

    # Space out the calls and increment counter
    time.sleep(0.12)
    i += 1
    print('\n')

bitcoin


ethereum




In [20]:
# Convert dictionary to cleaned up dataframe
gecko_price_mcap_vol_df = pd.DataFrame(price_mcap_vol_dict)
gecko_price_df = gecko_price_mcap_vol_df[gecko_price_mcap_vol_df.stat == 'usd_per_token']
gecko_price_df = gecko_price_df[~gecko_price_df.duplicated(subset=['gecko_id', 'date'])]
gecko_mcap_df  = gecko_price_mcap_vol_df[gecko_price_mcap_vol_df.stat == 'usd_mcap']
gecko_mcap_df  = gecko_mcap_df[~gecko_mcap_df.duplicated(subset=['gecko_id', 'date'])]
gecko_vol_df   = gecko_price_mcap_vol_df[gecko_price_mcap_vol_df.stat == 'usd_volume_24h']
gecko_vol_df   = gecko_vol_df[~gecko_vol_df.duplicated(subset=['gecko_id', 'date'])]
gecko_price_df = pd.pivot(gecko_price_df, index=['date', 'gecko_id'], columns='stat', values='value').reset_index()
gecko_mcap_df  = pd.pivot(gecko_mcap_df, index=['date', 'gecko_id'], columns='stat', values='value').reset_index()
gecko_vol_df   = pd.pivot(gecko_vol_df, index=['date', 'gecko_id'], columns='stat', values='value').reset_index()
gecko_df = gecko_price_df.merge(gecko_mcap_df,
                                on=['date', 'gecko_id'],
                                how='inner',
                                validate='one_to_one')
gecko_df = gecko_df.merge(gecko_vol_df,
                          on=['date', 'gecko_id'],
                          how='inner',
                          validate='one_to_one')

In [21]:
gecko_df

stat,date,gecko_id,usd_per_token,usd_mcap,usd_volume_24h
0,2015-12-01,bitcoin,358.820000,5.350374e+09,1.153668e+09
1,2015-12-01,ethereum,0.822734,6.184478e+07,4.942059e+05
2,2015-12-02,bitcoin,360.954000,5.383466e+09,1.149814e+09
3,2015-12-02,ethereum,0.824765,6.201881e+07,5.416811e+05
4,2015-12-03,bitcoin,362.767000,5.411894e+09,1.039913e+09
...,...,...,...,...,...
4885,2022-08-10,ethereum,1852.878555,2.221034e+11,2.169944e+10
4886,2022-08-11,bitcoin,23948.751704,4.580707e+11,3.389287e+10
4887,2022-08-11,ethereum,1881.427405,2.259735e+11,2.183707e+10
4888,2022-08-12,bitcoin,24411.379872,4.667657e+11,2.519519e+10


In [22]:
# Initialize a dictionary for the results
gecko_covars_dict = {'date':[],
                     'gecko_id': [],
                     'twitter_followers': [],
                     'reddit_average_posts_48h': [],
                     'reddit_subscribers': [],
                     'reddit_accounts_active_48h': [],
                     'subscribers': [],
                     'total_issues': [],
                     'closed_issues': [],
                     'pull_requests_merged': [],
                     'pull_request_contributors': [],
                     'code_additions_4_weeks': [],
                     'code_deletions_4_weeks': [],
                     'commit_count_4_weeks': []}

In [26]:

# Loop over all the tokens
for gecko_id in gecko_ids:
    # Update where we at
    print(gecko_id)
    
    # Extract the earliest date that we have price data for that token
    token_dates = gecko_df[gecko_df.gecko_id == gecko_id].date.values
    token_dates = [dt.datetime.strptime(date, "%Y-%m-%d").date() for date in token_dates]
    first_date  = np.min(token_dates)
    first_date  = first_date.strftime(format="%d-%m-%Y")

    # Build list of dates to pull
    dates = list(pd.date_range(first_date, '14-08-2022', freq='D').strftime('%d-%m-%Y'))

    # Loop over all the dates
    for date_to_call in dates:
        # Set up the call
        endpoint       = '/coins/' + gecko_id + '/history'
        final_url      = base_url+endpoint
        params         = base_params.copy()
        params['id']   = gecko_id 
        params['date'] = date_to_call

        # Make the call while checking it is successful
        nb_tries = 5
        while True:
            nb_tries -= 1
            try:
                r = requests.get(final_url, params=params, timeout=2)
                if r.status_code == 200:
                    r_json = r.json()
                    break
                else:
                    if nb_tries <= 0:
                        print(r.json())
                        assert(1==0),('some other error')
                    time.sleep(2)
                    pass
            except (ConnectionError, requests.exceptions.Timeout) as err:
                if nb_tries <= 0:
                    raise err
                else:
                    print('connection or timeout error')
                    time.sleep(5)

        # Add data to dictionary
        date_of_call = date_to_call[-4:] + '-' + date_to_call[3:5] + '-' + date_to_call[:2]
        gecko_covars_dict['date'].append(date_of_call)
        gecko_covars_dict['gecko_id'].append(gecko_id)

        if 'community_data' in r_json.keys():
            gecko_covars_dict['twitter_followers'].append(r_json['community_data']['twitter_followers'])
            gecko_covars_dict['reddit_average_posts_48h'].append(r_json['community_data']['reddit_average_posts_48h'])
            gecko_covars_dict['reddit_subscribers'].append(r_json['community_data']['reddit_subscribers'])
            gecko_covars_dict['reddit_accounts_active_48h'].append(r_json['community_data']['reddit_accounts_active_48h'])
        else:
            gecko_covars_dict['twitter_followers'].append(None)
            gecko_covars_dict['reddit_average_posts_48h'].append(None)
            gecko_covars_dict['reddit_subscribers'].append(None)
            gecko_covars_dict['reddit_accounts_active_48h'].append(None)

        if 'developer_data' in r_json.keys():
            gecko_covars_dict['subscribers'].append(r_json['developer_data']['subscribers'])
            gecko_covars_dict['total_issues'].append(r_json['developer_data']['total_issues'])
            gecko_covars_dict['closed_issues'].append(r_json['developer_data']['closed_issues'])
            gecko_covars_dict['pull_requests_merged'].append(r_json['developer_data']['pull_requests_merged'])
            gecko_covars_dict['pull_request_contributors'].append(r_json['developer_data']['pull_request_contributors'])
            gecko_covars_dict['code_additions_4_weeks'].append(r_json['developer_data']['code_additions_deletions_4_weeks']['additions'])
            gecko_covars_dict['code_deletions_4_weeks'].append(r_json['developer_data']['code_additions_deletions_4_weeks']['deletions'])
            gecko_covars_dict['commit_count_4_weeks'].append(r_json['developer_data']['commit_count_4_weeks'])
        else:
            gecko_covars_dict['subscribers'].append(None)
            gecko_covars_dict['total_issues'].append(None)
            gecko_covars_dict['closed_issues'].append(None)
            gecko_covars_dict['pull_requests_merged'].append(None)
            gecko_covars_dict['pull_request_contributors'].append(None)
            gecko_covars_dict['code_additions_4_weeks'].append(None)
            gecko_covars_dict['code_deletions_4_weeks'].append(None)
            gecko_covars_dict['commit_count_4_weeks'].append(None)

        # Space out the calls and increment counter
        time.sleep(1)
    
    # Update the counter for the tokens
    i += 1
    print('\n')


bitcoin
{'status': {'error_code': 429, 'error_message': "You've exceeded the Rate Limit. Please visit https://www.coingecko.com/en/api/pricing to subscribe to our API plans for higher rate limits."}}


AssertionError: some other error

In [ ]:
# Convert to a dataframe
gecko_covars_df = pd.DataFrame(gecko_covars_dict)
gecko_covars_df = gecko_covars_df[~gecko_covars_df.duplicated(subset=['date', 'gecko_id'])]

In [ ]:
# Merge the two dataframes together
df = gecko_df.merge(gecko_covars_df,
                    on=['date', 'gecko_id'],
                    how='outer',
                    validate='one_to_one')

In [ ]:
# OBTAIN THE MACRO DATA

gecko_macro_dict = {'date':[],
                    'total_mcap': [],
                    'total_volume': [],
                    'total_markets': []}

# Build list of dates to pull
dates = list(pd.date_range('01-12-2015', '28-07-2022', freq='D').strftime('%d-%m-%Y'))

# Loop over all the dates
for date_to_call in dates:
    # Set up the call
    endpoint       = '/global'
    final_url      = base_url+endpoint
    params         = base_params.copy()
    params['id']   = gecko_id 
    params['date'] = date_to_call

    # Make the call while checking it is successful
    nb_tries = 5
    while True:
        nb_tries -= 1
        try:
            r = requests.get(final_url, params=params, timeout=2)
            if r.status_code == 200:
                r_json = r.json()
                break
            else:
                if nb_tries <= 0:
                    assert(1==0),('some other error')
                time.sleep(2)
                pass
        except (ConnectionError, requests.exceptions.Timeout) as err:
            if nb_tries <= 0:
                raise err
            else:
                print('connection or timeout error')
                time.sleep(5)

    # Add data to dictionary
    date_of_call = date_to_call[-4:] + '-' + date_to_call[3:5] + '-' + date_to_call[:2]
    gecko_covars_dict['date'].append(date_of_call)
    gecko_covars_dict['gecko_id'].append(gecko_id)

    if 'community_data' in r_json.keys():
        gecko_covars_dict['twitter_followers'].append(r_json['community_data']['twitter_followers'])
    else:
        gecko_covars_dict['twitter_followers'].append(None)

    # Space out the calls and increment counter
    time.sleep(0.1)


In [ ]:
# Save the data
df.to_csv('../3-data/raw/coingecko_panel.csv', index=False)